<a href="https://colab.research.google.com/github/31adityakumar/StaySphere/blob/main/bookingAssistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import files

# Upload the file
uploaded = files.upload()

# Verify upload
!ls


Saving hotel_data.json to hotel_data.json
hotel_data.json  sample_data


In [ ]:
import json

# Load data
with open('hotel_data.json', 'r') as file:
    hotel_data = json.load(file)

# Prepare data in a conversational format
training_data = [{"text": f"Hotel: {hotel['hotelName']}\nRoom Details: {hotel['rooms']}\n"}
                 for hotel in hotel_data]

# Save as a text dataset
with open('training_data.txt', 'w') as f:
    for item in training_data:
        f.write(item['text'] + "\n")


In [ ]:
from transformers import BloomTokenizerFast, BloomForCausalLM

# Load tokenizer and model
tokenizer = BloomTokenizerFast.from_pretrained("bigscience/bloom-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load text dataset
dataset = load_dataset("text", data_files={"train": "training_data.txt"})

# Tokenize with error handling
def tokenize_function(examples):
    try:
        return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)
    except Exception as e:
        print(f"Tokenization error: {e}")
        return None

# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])



Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
# Validate tokenized dataset
for example in tokenized_datasets['train']:
    if len(example['input_ids']) != 512:
        print("Invalid tokenized entry:", example)


In [ ]:
print(tokenized_datasets)


DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 33
    })
})


In [ ]:
split_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)

# Rename the splits
tokenized_datasets = {
    "train": split_datasets["train"],
    "validation": split_datasets["test"],
}

In [ ]:
# Inspect dataset
for example in dataset['train']:
    if not example['text']:
        print("Empty entry found:", example)


Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}
Empty entry found: {'text': ''}


In [ ]:
# Remove empty entries
dataset = dataset.filter(lambda x: len(x['text']) > 0)


Filter:   0%|          | 0/33 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer
import torch

# Subclass Trainer to override compute_loss
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Extract labels or create them
        labels = inputs.pop("labels", None)
        if labels is None:
            # Create labels by shifting the input_ids
            labels = inputs["input_ids"].clone()
            labels[:, :-1] = inputs["input_ids"][:, 1:].clone()  # Shift labels for causal language modeling
            labels[:, -1] = -100  # Mask the last token (no loss on the last token)

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits

        # Compute loss
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Use CustomTrainer for training
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

# Start fine-tuning
trainer.train()


Step,Training Loss
10,1.054900
20,0.034200
30,0.024500
40,0.023400


Step,Training Loss
10,1.054900
20,0.034200
30,0.024500
40,0.023400
50,0.019200
60,0.015500


TrainOutput(global_step=65, training_loss=0.1816952021076129, metrics={'train_runtime': 1335.2159, 'train_samples_per_second': 0.097, 'train_steps_per_second': 0.049, 'total_flos': 120731908177920.0, 'train_loss': 0.1816952021076129, 'epoch': 5.0})

In [36]:
# Save model
output_dir = "./bloom_finetuned"

model.save_pretrained("./bloom_finetuned")
tokenizer.save_pretrained("./bloom_finetuned")

print(f"Model and tokenizer saved to {output_dir}")


Model and tokenizer saved to ./bloom_finetuned


In [37]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the saved model and tokenizer
model = AutoModelForCausalLM.from_pretrained(output_dir)
tokenizer = AutoTokenizer.from_pretrained(output_dir)

# Test the loaded model
input_text = "Welcome to our hotel, how can I help you?"
input_ids = tokenizer.encode(input_text, return_tensors="pt")
outputs = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    temperature=0.7,
    top_k=50,
    top_p=0.9
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Welcome to our hotel, how can I help you?', 'Deluxe Room', 'Deluxe Room', 'Deluxe Room', 'Deluxe Room', 'Deluxe Room', 'Deluxe Room', '
